In [2]:
from transformer import *

/Users/beesamprajveenkumar/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tgt_vocab_size = 5000
d_model = 256
num_heads = 4
num_layers = 4
d_ff = 512
max_seq_length = 100
dropout = 0.1

In [4]:
# Load the trained model
model = Transformer(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
model.load_state_dict(torch.load("v2.pth"))
model.eval()

Transformer(
  (decoder_embedding): Embedding(5000, 256)
  (positional_encoding): PositionalEncoding()
  (decoder_layers): ModuleList(
    (0-3): 4 x DecoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=256, out_features=256, bias=True)
        (W_k): Linear(in_features=256, out_features=256, bias=True)
        (W_v): Linear(in_features=256, out_features=256, bias=True)
        (W_o): Linear(in_features=256, out_features=256, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=256, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (fc): Linear(in_features=256, out_features=5000, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [5]:
# Example usage
sp = spm.SentencePieceProcessor()
sp.load('small_m.model')

True

In [9]:
def word_to_token_id(word, sp_model):
    # Convert word to token ID using SentencePiece
    return sp_model.piece_to_id(word)

def generate_text(model, sp_model, starting_word, ending_word, max_length):
    
    # Convert starting and ending words to token IDs
    starting_token_id = word_to_token_id(starting_word, sp_model)
    if starting_token_id is None:
        raise ValueError(f"Starting word '{starting_word}' not found in vocabulary.")
    
    ending_token_id = word_to_token_id(ending_word, sp_model)
    if ending_token_id is None:
        raise ValueError(f"Ending word '{ending_word}' not found in vocabulary.")
    
    
    generated_sequence = [starting_token_id]
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor([generated_sequence])
            output = model(input_tensor)
            predicted_token = output.argmax(-1)[:,-1].item()
            generated_sequence.append(predicted_token)
            if predicted_token == ending_token_id:
                break
            
    # Convert token IDs to words using SentencePiece
    generated_text = sp_model.decode_ids(generated_sequence)
    return generated_text

# Example usage:
starting_word = "<sos>"
ending_word = "</sos>"
max_length = 100
generated_sequence = generate_text(model, sp, starting_word, ending_word, max_length)
print("Generated sequence:", generated_sequence)

Generated sequence:  ⁇  lily liked to play with her toys. she liked to play with her toys and run. she liked to her room and run and run around outside. she saw a big, green grass. she wanted to play with her room the doll. she ran to the doll and saw a big, but her mom said, "no, lily, lily. i want to clean the doll. it's mine!" lily said, "no, lily. i want to clean the doll, but you can
